In [ ]:
import yaml
import requests

# Download the YAML file
file_url = "https://raw.githubusercontent.com/NFDI4BIOIMAGE/training/main/resources/nfdi4bioimage.yml"
response = requests.get(file_url)

# Parse YAML
data = yaml.safe_load(response.content)

# Count all entries and those without license
total_entries = 0
entries_without_license = []
entries_with_license = []

for entry in data.get("resources", []):
    total_entries += 1
    if not entry.get("license"):
        entries_without_license.append(entry.get("url", "No URL"))
    else:
        entries_with_license.append((entry.get("url", "No URL"), entry.get("license")))

print(f"Total entries: {total_entries}")
print(f"Entries without license: {len(entries_without_license)}")
print(f"Entries with license: {len(entries_with_license)}")
print("\nEntries without license:")
for url in entries_without_license:
    print(f"- {url}")
print("\nEntries with license:")
for url, license in entries_with_license:
    print(f"- {url} (License: {license})")




Total entries: 756
Entries without license: 76
Entries with license: 680

Entries without license:
- https://focalplane.biologists.com/2023/07/26/sharing-your-poster-on-figshare/
- https://datamanagement.hms.harvard.edu/news/promoting-data-management-nikon-imaging-center-and-cell-biology-microscopy-facility
- https://blog.delmic.com/data-handling-in-large-scale-electron-microscopy
- https://focalplane.biologists.com/2023/06/01/tracking-in-napari/
- https://focalplane.biologists.com/2023/05/03/feature-extraction-in-napari/
- https://focalplane.biologists.com/2023/03/30/annotating-3d-images-in-napari/
- https://focalplane.biologists.com/2023/04/13/quality-assurance-of-segmentation-results/
- https://focalplane.biologists.com/2023/03/02/rescaling-images-and-pixel-anisotropy/
- https://forum.image.sc/t/user-friendly-image-metadata-annotation-tool-workflow-for-omero/87925/1
- https://focalplane.biologists.com/2024/04/03/how-to-write-a-bug-report/
- https://focalplane.biologists.com/2024/07/

In [ ]:
# Count all entries without a 'type' tag and count those that are Zenodo PDF links to actual PDF files
#TODO: make sure that pds are slides and not text files
from caching import extract_zenodo_ids, get_zenodo_pdfs

total_entries = 0
entries_without_type = []
entries_with_type = []
entries_without_type_zenodo_pdf = []
test = []

for entry in data.get("resources", []):
    total_entries += 1
    url = entry.get("url", "No URL")
    if not entry.get("type"):
        entries_without_type.append(url)
        zenodo_ids = extract_zenodo_ids(url)
        if zenodo_ids:
            # Check if the Zenodo record has at least one PDF file
            for record_id in zenodo_ids:
                pdf_files = get_zenodo_pdfs(record_id)
                if pdf_files:
                    entries_without_type_zenodo_pdf.append(url)
                    break
    else:
        entries_with_type.append((url, entry.get("type")))

print(f"Total entries: {total_entries}")
print(f"Entries with type: {len(entries_with_type)}")
print(f"Entries without type: {len(entries_without_type)}")
print(f"Entries without type that are Zenodo PDF links: {len(entries_without_type_zenodo_pdf)}")

print("\nEntries without type that are Zenodo PDF links:")
for url in entries_without_type_zenodo_pdf:
    print(f"- {url}")

Failed to fetch Zenodo record 14729452, skipping...
Failed to fetch Zenodo record 14845059, skipping...
Failed to fetch Zenodo record 14832855, skipping...
Failed to fetch Zenodo record 13880367, skipping...
Failed to fetch Zenodo record 14917722, skipping...
Failed to fetch Zenodo record 14913673, skipping...
Failed to fetch Zenodo record 14893791, skipping...
Failed to fetch Zenodo record 14053758, skipping...
Failed to fetch Zenodo record 14043615, skipping...
Failed to fetch Zenodo record 14937632, skipping...
Failed to fetch Zenodo record 14933318, skipping...
Failed to fetch Zenodo record 14944040, skipping...
Failed to fetch Zenodo record 15031842, skipping...
Failed to fetch Zenodo record 15026373, skipping...
Failed to fetch Zenodo record 14988921, skipping...
Failed to fetch Zenodo record 14979253, skipping...
Failed to fetch Zenodo record 14975462, skipping...
Failed to fetch Zenodo record 14909526, skipping...
Failed to fetch Zenodo record 15001649, skipping...
Failed to fe

In [ ]:
# RAW CELL: 
# try for later 
# Check for URLs that point directly to a .pdf file
# man könnte auch noch abfangen ob von github etc.
# if isinstance(url, str) and url.lower().endswith(".pdf"):
#     entries_without_type_zenodo_pdf.append(url)
#     test.append(url)

### Caching the Embeddings for each Presentation Slide in our Training Material 
For each Zenodo Record in our Resources "nfdi4bioimage.yml" File, we can generate different embeddings (visual, text and mixed). 
Instead of Calculating it over and over again for different tasks, we can calculate the Embeddings once and store them somewhere (e.g. via Huggingface) to load them again at any time.
For now, the Embeddings are stored as a Dataset on Huggingface.

To get started, you have to choose between using the free Github Models or your own API key (from OpenAI). For that, adjust the variables __use_gh_models__ and __use_openai__ in the cell below. If neccessary, also edit the name of the API Key (as stored in your environment).

In [1]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from huggingface_hub import login
import os

#Authenticate your current session
login(token=os.getenv("HF_TOKEN"))

In [ ]:
# Decide on whether to use GH Models or OpenAI API
use_gh_models = True
use_openai = False

if use_gh_models:
    token = os.environ["GITHUB_TOKEN"]
    
if use_openai:
    token = os.environ["OPENAI_API_KEY"]

1. Create lists of valid licenses to filter out unwanted entries

In [7]:
valid_licenses = [
    'cc-by-3.0',
    'mozilla public license 2.0',
    'cc0 1.0 universal',
    'cc0-1.0',
    'apache-2.0',
    'bsd 3-clause',
    'mit-license',
    'odc-by-1.0',
    'apache license 2.0',
    'cc-by-3.0 unported',
    'cc-by-nc-3.0 unported',
    'cc0',
    'bsd3-clause',
    'cc-zero',
    'public domain',
    'mit license',
    'bsd 3-clause "new" or "revised" license',
    'cc0 (mostly, but can differ depending on resource)',
    'cc-by-4.0 international',
    'bsd-3-clause',
    'cc-by-nc-4.0',
    'creative commons attribution 4.0 international',
    'mit',
    'cc-by-4.0',
    'bsd-2-clause',
    'academic free license version 3.0',
    'creative commons attribution 3.0 (cc by 3.0) license',
    'cc-by-4.0',
    'bsd-3-clause',
]

In [5]:
unclear_licenses = ['custom license', 'unlicensed', 'nan', 'none', 'unknown', 'other-open', 'unkown']

2. Load all Zenodo Record IDs from our Training Material

In [6]:
from caching import get_zenodo_ids_from_yaml
import requests

file_url = "https://raw.githubusercontent.com/NFDI4BIOIMAGE/training/main/resources/nfdi4bioimage.yml"
yaml_file = "nfdi4bioimage.yml" 
response = requests.get(file_url)

# Download the current Training Material yaml file from the Git Repository
with open(yaml_file, "wb") as file:
    file.write(response.content)
print(f"File downloaded successfully as {yaml_file}")

# Extract the Zenodo Record IDs
zenodo_ids = get_zenodo_ids_from_yaml(yaml_file, valid_licenses, unclear_licenses)
print(f"Found {len(zenodo_ids)} Zenodo records: {zenodo_ids}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


File downloaded successfully as nfdi4bioimage.yml
Found 39 Zenodo records: ['10008464', '10008465', '10083555', '10654775', '10679054', '10687658', '10793699', '10815329', '10816895', '10886749', '10939519', '10942559', '10970869', '10972692', '10990107', '11031746', '11066250', '11107798', '11265038', '11396199', '11472148', '11474407', '11548617', '12623730', '3778431', '4317149', '4328911', '4330625', '4334697', '4461261', '4630788', '4748510', '4748534', '4778265', '8070038', '8323588', '8329305', '8329306', '8414318']


3. Calculate and save all embeddings in a Huggingface Dataset

In [3]:
from caching import ensure_repo_exists, load_cache_dataset, append_rows_to_dataset, embed_text, embed_visual, embed_mixed, cache_hf
from datasets import concatenate_datasets, load_dataset

repo_name = "ScaDS-AI/SlightInsight_Cache"
for record_id in zenodo_ids:
    cache_hf(record_id, token, use_openai, repo_name)

Repository 'haesleinhuepf/SlightInsight_Cache' created.
Processing 2023-Moore-N4BI-AHM-Welcome.pdf from Zenodo Record 10008464


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Finished processing Zenodo Record 10008464.
Repository 'haesleinhuepf/SlightInsight_Cache' already exists.


README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/2600 [00:00<?, ? examples/s]

Processing 20231005_PIC_UFZ_NFDI4BIOIMAGE_Massei_public.pdf from Zenodo Record 8414318


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Finished processing Zenodo Record 8414318.


4. Example on how to load the data again for a specific slide from a specific Record/Presentation

In [2]:
from caching import load_single_hf_cache
import pandas as pd

df = load_single_hf_cache("10083555", 3)
df

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


,key,zenodo_record_id,zenodo_filename,page_number,text_embedding,visual_embedding,mixed_embedding,extracted_text
0,record10083555_pdf1_slide3,10083555,Bio-Image_Data_Strudel_TU-Dresden_TP_Workshop_...,3,"[-0.8083789, -0.57003796, 0.026535584, 0.15640...","[-0.002609865041449666, -0.11352983117103577, ...","[-0.53784376, -0.4322776, 0.20752871, -0.15392...",What is the ‚Nationale Forschungsdaten Infrast...


5. How to load the whole dataset

In [3]:
from caching import load_full_hf_cache

df_full = load_full_hf_cache()    
df_full.head()

,key,zenodo_record_id,zenodo_filename,page_number,text_embedding,visual_embedding,mixed_embedding,extracted_text
0,record10008464_pdf1_slide1,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,1,"[0.8054575, -0.4204579, 0.111390926, 0.2636367...","[0.22024887800216675, 0.6564452648162842, 0.02...","[0.22330284, -0.5643485, 0.32713842, -0.074667...","Welcome to\nBioImage Town!\nJosh Moore, Senior..."
1,record10008464_pdf1_slide2,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,2,"[1.1522328, 0.02467385, 0.23145455, 0.17258961...","[-0.3846272826194763, -0.01668522134423256, -0...","[0.34757608, -0.6562839, 0.5335755, -0.1906935...",Special welcome\nHonored New\nGuests Colleague...
2,record10008464_pdf1_slide3,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,3,"[1.0057557, 0.18344171, 0.03796136, 0.23549518...","[-0.32376205921173096, 0.06897055357694626, 0....","[0.06456853, -0.79703176, 0.871621, -0.8768049...",Value of bioimages\nhttps://ome-model.readthed...
3,record10008464_pdf1_slide4,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,4,"[1.2563236, 0.25261688, 0.0040982994, 0.202414...","[-0.03493745997548103, -0.11242333054542542, 0...","[0.3643795, -0.40593308, 0.97395766, -0.495041...",Value of bioimages\nAll-Hands Meeting · Josh M...
4,record10008464_pdf1_slide5,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,5,"[0.8471789, -0.40933934, 0.85754687, -0.496454...","[-0.6693567037582397, -0.02434205450117588, -0...","[-0.22939722, -0.6752343, 0.57570195, 0.046955...","Image Data Management isn’t always FAIR\n""Clar..."
